In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir, rename
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam_v2
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet import ResNet101
from keras.applications.mobilenet_v3 import MobileNetV3Small

In [ ]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = "gdrive/MyDrive/MLProjs/PlantDiseases"
width=256
height=256
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(f"{image_dir}")
        if image is not None:
            image = cv2.resize(image, default_image_size)
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = list(), list()
try:
    print("[INFO] Loading images ...")
    dataset_root = f"{directory_root}/Dataset"

    plant_disease_folder_list = listdir(f"{directory_root}/Dataset")

    for disease_folder in plant_disease_folder_list:
        if disease_folder == ".DS_Store":
            plant_disease_folder_list.remove(disease_folder)

    for plant_disease_folder in plant_disease_folder_list:
        print(f"[INFO] Processing {plant_disease_folder} ...")
        plant_disease_image_list = listdir(f"{dataset_root}/{plant_disease_folder}/")

        for single_plant_disease_image in plant_disease_image_list:
            if single_plant_disease_image == ".DS_Store":
                plant_disease_image_list.remove(single_plant_disease_image)

        for image in plant_disease_image_list[:100]:
            image_directory = f"{dataset_root}/{plant_disease_folder}/{image}"
            if image_directory.endswith(".jpg") or image_directory.endswith(".JPG"):
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")
            
except Exception as e:
    print(f"Error : {e}")

In [ ]:
# Load the data
np_image_list = np.load(f"{directory_root}/imageArray.npy")

with open(f"{directory_root}/labels.txt", "r") as f:
  label_list = [x.strip() for x in f]

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
with open("label_transform.pkl", "wb") as f:
    pickle.dump((label_binarizer), f)
n_classes = len(label_binarizer.classes_)

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16)
with open(f"{directory_root}/imageArray.npy", "wb") as f:
  np.save(f, np_image_list)

with open(f"{directory_root}/labels.txt", "w") as f:
  print("\n".join(label_list), file=f, end="")
del image_list
del label_list

In [ ]:
print("[INFO] Splitting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state=42)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
# CNN Custom
# model = Sequential()
# inputShape = (height, width, depth)
# chanDim = -1
# if K.image_data_format() == "channels_first":
#     inputShape = (depth, height, width)
#     chanDim = 1

# model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))

# model.add(Conv2D(64, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))

# model.add(Conv2D(128, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(BatchNormalization(axis=chanDim))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(1024))
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(n_classes))
# model.add(Activation("softmax"))

In [ ]:
model = Sequential()
inputShape = (height, width, depth)

# VGG16
# model.add(VGG16(
#     input_shape=inputShape,
#     include_top=False,
#     weights="imagenet"
# ))

# InceptionV3
# model.add(InceptionV3(
#     input_shape = inputShape,
#     include_top = False,
#     weights = "imagenet"
# ))

# ResNet101
# model.add(ResNet101(
#     input_shape = inputShape,
#     include_top = False,
#     weights = "imagenet"
# ))

# MobileNetV3Small
model.add(MobileNetV3Small(
    input_shape = inputShape,
    include_top = False,
    weights = "imagenet"
))

for layer in model.layers:
  layer.trainable = False

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))
model.summary()

In [ ]:
opt = adam_v2.Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 85,
    epochs=EPOCHS,
    verbose=1
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

Results:<br>
VGG16 --> >95% <br>
InceptionV3 --> ~44.8% <br>
ResNet101 --> 94.7% <br>
MobileNetV3Small --> 91.6%